[辻真吾・矢吹太朗『ゼロからはじめるデータサイエンス入門』（講談社, 2021）](https://github.com/taroyabuki/fromzero)

In [ ]:
# Google Colaboratoryの環境設定
if (Sys.getenv("COLAB_JUPYTER_IP") != "") {
  options(Ncpus = parallel::detectCores())
  installed_packages <- rownames(installed.packages())
  packages_to_install <- c("h2o", "keras3")
  install.packages(setdiff(packages_to_install, installed_packages))
}

## 11.1 Kerasによる回帰

In [ ]:
if (Sys.getenv("COLAB_JUPYTER_IP") != "") {
  library(keras3)
} else {
  library(keras)
}
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
tmp <- read_csv(my_url)

In [ ]:
my_data <- tmp[sample(nrow(tmp)), ]

In [ ]:
X <- my_data %>%
  select(-LPRICE2) %>% scale
y <- my_data$LPRICE2

In [ ]:
curve(activation_relu(x), -3, 3)

In [ ]:
my_model <- keras_model_sequential() %>%
  layer_dense(units = 3, activation = "relu", input_shape = c(4)) %>%
  layer_dense(units = 1)

summary(my_model) # ネットワークの概要
# 割愛（Pythonの結果を参照）

In [ ]:
my_model %>% compile(
  loss = "mse",
  optimizer = "rmsprop")

In [ ]:
my_cb <- callback_early_stopping(
  patience = 20,
  restore_best_weights = TRUE)

In [ ]:
my_history <- my_model %>% fit(
    x = X,
    y = y,
    validation_split = 0.25,
    batch_size = 10,
    epochs = 500,
    callbacks = list(my_cb),
    verbose = 0)

In [ ]:
plot(my_history)

In [ ]:
my_history

In [ ]:
y_ <- my_model %>% predict(X)
mean((y_ - y)^2)**0.5

## 11.2 Kerasによる分類

In [ ]:
if (Sys.getenv("COLAB_JUPYTER_IP") != "") {
  library(keras3)
} else {
  library(keras)
}
library(tidyverse)
my_data <- iris[sample(nrow(iris)), ]

In [ ]:
X <- my_data %>%
  select(-Species) %>% scale
y <- as.integer(my_data$Species) - 1

In [ ]:
my_model <- keras_model_sequential() %>%
  layer_dense(units = 3, activation = "relu", input_shape = c(4)) %>%
  layer_dense(units = 3, activation = "softmax")

In [ ]:
my_model %>% compile(loss = "sparse_categorical_crossentropy",
                     optimizer = "rmsprop",
                     metrics = c("accuracy"))

In [ ]:
my_cb <- callback_early_stopping(
  patience = 20,
  restore_best_weights = TRUE)

my_history <- my_model %>%
  fit(x = X,
    y = y,
    validation_split = 0.25,
    batch_size = 10,
    epochs = 500,
    callbacks = list(my_cb),
    verbose = 0)

plot(my_history)

In [ ]:
my_history

In [ ]:
tmp <- my_model %>% predict(X)
y_ <- apply(tmp, 1, which.max) - 1
mean(y_ == y)

In [ ]:
-mean(log(c(0.8, 0.7, 0.3, 0.8)))

-mean(log(c(0.7, 0.6, 0.2, 0.7)))

In [ ]:
y <- c(2, 1, 0, 1)
y_1 <- rbind(c(0.1, 0.1, 0.8),
             c(0.1, 0.7, 0.2),
             c(0.3, 0.4, 0.3),
             c(0.1, 0.8, 0.1))
y_2 <- rbind(c(0.1, 0.2, 0.7),
             c(0.2, 0.6, 0.2),
             c(0.2, 0.5, 0.3),
             c(0.2, 0.7, 0.1))

In [ ]:
c(mean(as.array(loss_sparse_categorical_crossentropy(y_true = y, y_pred = y_1))),
  mean(as.array(loss_sparse_categorical_crossentropy(y_true = y, y_pred = y_2))))

## 11.3 MNIST：手書き数字の分類

In [ ]:
if (Sys.getenv("COLAB_JUPYTER_IP") != "") {
  library(keras3)
} else {
  library(keras)
}
library(tidyverse)
c(c(x_train, y_train), c(x_test, y_test)) %<-% dataset_mnist()

In [ ]:
dim(x_train)

In [ ]:
x_train[5, , ]

In [ ]:
plot(as.raster(x = x_train[5, , ],
               max = max(x_train)))

In [ ]:
head(y_train)

In [ ]:
c(min(x_train), max(x_train))

In [ ]:
x_train <- x_train / 255
x_test  <- x_test  / 255

In [ ]:
my_index <- sample(1:60000, 6000)
x_train <- x_train[my_index, , ]
y_train <- y_train[my_index]

In [ ]:
my_model <- keras_model_sequential() %>%
  layer_flatten(input_shape = c(28, 28)) %>%
  layer_dense(units = 256, activation = "relu") %>%
  layer_dense(units = 10, activation = "softmax")
summary(my_model)
# 割愛（Pythonの結果を参照）

my_model %>% compile(loss = "sparse_categorical_crossentropy",
                     optimizer = "rmsprop",
                     metrics = c("accuracy"))

my_cb <- callback_early_stopping(patience = 5,
                                 restore_best_weights = TRUE)

In [ ]:
my_history <- my_model %>%
  fit(x = x_train,
      y = y_train,
      validation_split = 0.2,
      batch_size = 128,
      epochs = 20,
      callbacks = list(my_cb),
      verbose = 0)

plot(my_history)

In [ ]:
tmp <- my_model %>% predict(x_test)
y_ <- apply(tmp, 1, which.max) - 1
table(y_, y_test)

In [ ]:
mean(y_ == y_test)

In [ ]:
my_model %>%
  evaluate(x = x_test, y = y_test)

In [ ]:
x_train2d <- x_train %>% array_reshape(c(-1, 28, 28, 1))
x_test2d  <- x_test  %>% array_reshape(c(-1, 28, 28, 1))

In [ ]:
my_model <- keras_model_sequential() %>%
  layer_conv_2d(filters = 32, kernel_size = 3,  # 畳み込み層
                activation = "relu",
                input_shape = c(28, 28, 1)) %>%
  layer_max_pooling_2d(pool_size = 2) %>%       # プーリング層
  layer_flatten() %>%
  layer_dense(units = 128, activation = "relu") %>%
  layer_dense(units = 10, activation = "softmax")

summary(my_model)
# 割愛（Python の結果を参照）

my_model %>% compile(loss = "sparse_categorical_crossentropy",
                     optimizer = "rmsprop",
                     metrics = c("accuracy"))

my_cb <- callback_early_stopping(patience = 5,
                                 restore_best_weights = TRUE)

In [ ]:
my_history <- my_model %>%
  fit(x = x_train2d,
      y = y_train,
      validation_split = 0.2,
      batch_size = 128,
      epochs = 20,
      callbacks = list(my_cb),
      verbose = 0)

plot(my_history)

In [ ]:
my_model %>%
  evaluate(x = x_test2d, y = y_test)

In [ ]:
my_model <- keras_model_sequential() %>%
  layer_conv_2d(filters = 20, kernel_size = 5, activation = "relu",
                input_shape = c(28, 28, 1)) %>%
  layer_max_pooling_2d(pool_size = 2, strides = 2) %>%
  layer_conv_2d(filters = 50, kernel_size = 5, activation = "relu") %>%
  layer_max_pooling_2d(pool_size = 2, strides = 2) %>%
  layer_dropout(rate = 0.25) %>%
  layer_flatten() %>%
  layer_dense(units = 500, activation = "relu") %>%
  layer_dropout(rate = 0.5) %>%
  layer_dense(units = 10, activation = "softmax")

my_model %>% compile(
  loss = "sparse_categorical_crossentropy",
  optimizer = "rmsprop",
  metrics = c("accuracy"))

my_cb <- callback_early_stopping(patience = 5,
                                 restore_best_weights = TRUE)

In [ ]:
my_history <- my_model %>%
  fit(x = x_train2d,
      y = y_train,
      validation_split = 0.2,
      batch_size = 128,
      epochs = 20,
      callbacks = list(my_cb),
      verbose = 0)

plot(my_history)

In [ ]:
my_model %>%
  evaluate(x = x_test2d, y = y_test)

In [ ]:
y_prob <- my_model %>% predict(x_test2d) # カテゴリに属する確率

my_result <- data.frame(
  y_prob = apply(y_prob, 1, max),           # 確率の最大値
  y_     = apply(y_prob, 1, which.max) - 1, # 予測カテゴリ
  y      = y_test,                          # 正解
  id     = seq_len(length(y_test))) %>%     # 番号
  filter(y_ != y) %>%                       # 予測がはずれたものを残す
  arrange(desc(y_prob))                     # 確率の大きい順に並び替える

In [ ]:
head(my_result)

In [ ]:
tmp <- my_result[1:5, ]$id
my_labels <- sprintf("%s (%s)",
  my_result[1:5, ]$y, tmp)
my_fig <- expand.grid(
  label = my_labels,
  y = 28:1,
  x = 1:28)
my_fig$z <- as.vector(
  x_test[tmp, , ])

my_fig %>% ggplot(
  aes(x = x, y = y, fill = z)) +
  geom_raster() +
  coord_fixed() +
  theme_void() +
  theme(legend.position = "none") +
  facet_grid(. ~ label)

## 11.4 AutoML

In [ ]:
library(h2o)
if (Sys.getenv("COLAB_JUPYTER_IP") != "") {
  library(keras3)
} else {
  library(keras)
}
library(tidyverse)

h2o.init()
h2o.no_progress()
# h2o.shutdown(prompt = FALSE) # 停止

In [ ]:
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)
my_frame <- as.h2o(my_data) # 通常のデータフレームをH2OFrameに変換する．
# あるいは
my_frame <- h2o.importFile(my_url, header = TRUE) # データを読み込む．

In [ ]:
my_frame

# 通常のデータフレームに戻す．
my_frame %>% as.data.frame %>% head
# 結果は割愛（見た目は同じ）

In [ ]:
my_model <- h2o.automl(
    y = "LPRICE2",
    training_frame = my_frame,
    max_runtime_secs = 60)

In [ ]:
min(my_model@leaderboard$rmse)

In [ ]:
tmp <- my_model %>%
  predict(my_frame) %>%
  as.data.frame
y_ <- tmp$predict
y  <- my_data$LPRICE2

plot(y, y_)

In [ ]:
c(c(x_train, y_train), c(x_test, y_test)) %<-% dataset_mnist()
my_index <- sample(1:60000, 6000)
x_train <- x_train[my_index, , ]
y_train <- y_train[my_index]

In [ ]:
tmp <- x_train %>%
  array_reshape(c(-1, 28 * 28)) %>%
  as.data.frame
tmp$y <- as.factor(y_train)
my_train <- as.h2o(tmp)

tmp <- x_test %>%
  array_reshape(c(-1, 28 * 28)) %>%
  as.data.frame
my_test <- as.h2o(tmp)

In [ ]:
my_model <- h2o.automl(
    y = "y",
    training_frame = my_train,
    max_runtime_secs = 120)

In [ ]:
min(my_model@leaderboard$
    mean_per_class_error)

In [ ]:
tmp <- my_model %>%
  predict(my_test) %>% as.data.frame
y_ <- tmp$predict

mean(y_ == y_test)